In [ ]:
#| default_exp guppy

# guppy

To retrieve the current signals from the base-called fast5 file.

## Some Old Codes


---

This is the place for previous codes that retrieve the signal mapping results, which are not used in today's package:

>Because the speed is much slower than the newer one.

```python

def _makeup_fast5_read(signal: list, cumulative_moves: list, base_possibilities: np.array, sequence: str, qualities: str, block_stride: int):
    '''
    getting the current information of each base on the read

    [extended_summary]

    Args:
        signal (list): [description]
        cumulative_moves (list): [description]
        base_possibilities (np.array): [description]
        sequence (str): [description]
        qualities (str): [description]
        block_stride (int): [description]

    Returns:
        [type]: [description]
    '''
    ordered = OrderedDict()
    seq_length = len(sequence)
    prev = None
    for i in range(seq_length):
        signal_start = cumulative_moves.index(seq_length-i)*block_stride
        if i == 0:
            signal_end = len(cumulative_moves)*block_stride
        else:
            signal_end = prev
        ordered[i] = {
            'base': sequence[i],
            'quality': qualities[i],
            'signal': np.array(signal[signal_start:signal_end][::-1]),
            'base_possibility': base_possibilities[signal_start:signal_end][::-1]
        }
        prev = signal_start
    return ordered

def get_signal_chunk_guppy(read,latest:bool=True,group_id:str='000',scale:bool=True,test:bool=False,verbose:bool=False):
    '''
    get_signal_chunk_guppy _summary_

    _extended_summary_

    Args:
        read (_type_): _description_
        latest (bool, optional): _description_. Defaults to True.
        group_id (str, optional): _description_. Defaults to '000'.
        scale (bool, optional): _description_. Defaults to True.
        test (bool, optional): _description_. Defaults to False.
        verbose (bool, optional): _description_. Defaults to False.

    Returns:
        _type_: _description_
    '''
    # if using the latest analysis from the fast5 file 
    if latest:
        basecall_analysis = read.get_latest_analysis('Basecall_1D')
        segmentation_analysis = read.get_latest_analysis('Segmentation')
        group_id = basecall_analysis.replace('Basecall_1D_','')
    else:
        basecall_analysis = f'Basecall_1D_{group_id}'
        segmentation_analysis = f'Segmentation_{group_id}'

    # current signal, the scale should be true to extract the pA signal
    current_signal = read.get_raw_data(scale=scale)
    # move table from the guppy basecaller
    move_table = read.get_analysis_dataset(basecall_analysis,'BaseCalled_template/Move')
    
    if test:
        # trace_table may be not useful in final implements but it is useful for testing
        # get the base_possibilities from the guppy basecaller
        trace_table = read.get_analysis_dataset(basecall_analysis,'BaseCalled_template/Trace')

    segmentation_summary = read.get_summary_data(segmentation_analysis)
    basecall_summary = read.get_summary_data(basecall_analysis)
    first_sample_template = segmentation_summary['segmentation']['first_sample_template']
    duration_template = segmentation_summary['segmentation']['duration_template']
    block_stride = basecall_summary['basecall_1d_template']['block_stride']

    template_signal_end = first_sample_template+int(duration_template/block_stride)*block_stride

    # The Trace order is: A, C, G, U, A', C', G', U'.
    # The flip possibility is the first 4 columns, the flop possibility is the last 4 columns.
    
    def _overlap_sum(alist,number):
        return alist[0:number]+alist[number:2*number]
    base_pos = []
    for i in trace_table:
        base_pos.append(np.array(_overlap_sum(i/i.sum(),4)))
    base_possibility = np.array(base_pos).repeat(block_stride,axis=0)

    cumulative_moves = list(np.add.accumulate(move_table))

    # TODO: the extreme values in the signal, how to handle them? 
    # I asked it in the nanopore community: https://community.nanoporetech.com/posts/guppy-v5-0-16-patch-releas#comment_36772
    template_signals = current_signal[first_sample_template:template_signal_end]

    _, seq, _, qual, _ = read.get_analysis_dataset(basecall_analysis,'BaseCalled_template/Fastq').split('\n')

    return _makeup_fast5_read(template_signals,cumulative_moves,base_possibility,seq,qual,block_stride),current_signal,first_sample_template


def padding_signal(signal_dict,location,padding=0,filter_no_padding=False):
    '''
    extract signals from the signal_dict.
    Sometimes, we want to extract signals correspond to more than one nucleotide, we also want the signals of the padding bases. 

    [extended_summary]

    Args:
        signal_dict ([type]): [description]
        location ([type]): [description]
        padding (int, optional): [description]. Defaults to 0.
        filter_no_padding (bool, optional): [description]. Defaults to False.
    '''
    if padding ==0:
        return signal_dict[location]['signal'], [ len(signal_dict[location]['signal'])-1], [signal_dict[location]['base']]
    else:
        
        if filter_no_padding and ((location-padding <0) or (location+padding+1 not in signal_dict)):
            return None,None,None
        else:
            low_b = max(0,location-padding)
            up_b = min(location+padding+1,list(signal_dict.keys())[-1])
            signals = []
            seg = []
            seg_loc = 0
            bases = []
            for i in range(low_b,up_b):
                signal = signal_dict[i]['signal']
                signals.append(signal)
                seg_loc += len(signal)
                seg.append(seg_loc-1)
                bases.append(signal_dict[i]['base'])
    return np.concatenate(signals),seg,bases

```



In [ ]:
#| export

import numpy as np


In [ ]:
#| export

class GuppyCalledRead(object):
    '''
    GuppyCalledRead is a class that contains the following attributes:
    1. read: the fast5 read object
    2. basecall_analysis: the basecall analysis name
    3. segmentation_analysis: the segmentation analysis name
    4. group_id: the group id of the basecall analysis
    5. move_table: the move table of the basecall analysis
    6. segmentation_summary: the segmentation summary of the segmentation analysis
    7. basecall_summary: the basecall summary of the basecall analysis
    8. first_sample_template: the first sample template of the segmentation analysis
    9. block_stride: the block stride of the basecall analysis
    10. current_signal: the current signal of the read
    11. sequence_length: the sequence length of the read
    12. move_list: the move list of the read

    The following methods are available:
    1. loading_necessary(): load the necessary attributes of the read
    2. loading_base_possibility(): load the base possibility of the read
    3. loading_seq(): load the sequence of the read
    4. get_template_signal(): get the template signal of the read
    5. get_signal_of_seq_loc(start,end): get the signal of the seq loc

    Args:
        fast5_read: the fast5 read object
    '''
    def __init__(self, fast5_read,scale=True):
        self.read = fast5_read
        self.scale = scale
        self.loading_necessary()

    def loading_necessary(self):
        self.basecall_analysis = self.read.get_latest_analysis('Basecall_1D')
        self.segmentation_analysis = self.read.get_latest_analysis(
            'Segmentation')
        self.group_id = self.basecall_analysis.replace('Basecall_1D_', '')
        move_table = self.read.get_analysis_dataset(
            self.basecall_analysis, 'BaseCalled_template/Move')
        self.segmentation_summary = self.read.get_summary_data(self.segmentation_analysis)
        self.basecall_summary = self.read.get_summary_data(self.basecall_analysis)
        self.first_sample_template = self.segmentation_summary['segmentation']['first_sample_template']
        self.block_stride = self.basecall_summary['basecall_1d_template']['block_stride']
        self.current_signal = self.read.get_raw_data(scale=self.scale)[self.first_sample_template:].astype(np.double)
        self.sequence_length = self.basecall_summary['basecall_1d_template']['sequence_length']
        self._move_table_to_list(move_table)
    
    def _move_table_to_list(self,move_table):
        self.move_list = []
        index = -1
        move_table_list = move_table.tolist()
        while index < len(move_table):
            try:
                index = move_table_list.index(1, index+1)
                self.move_list.append(index*self.block_stride)
            except ValueError:
                self.move_list.append(move_table.shape[0]*self.block_stride)
                break

    def loading_base_possibility(self):
        self.trace_table = self.read.get_analysis_dataset(
            self.basecall_analysis, 'BaseCalled_template/Trace')
        # The Trace order is: A, C, G, U, A', C', G', U'.
        # The flip possibility is the first 4 columns, the flop possibility is the last 4 columns.
        def _overlap_sum(alist, number):
            return alist[0:number]+alist[number:2*number]
        base_pos = []
        for i in self.trace_table:
            base_pos.append(np.array(_overlap_sum(i/i.sum(), 4)))
        self.base_possibility = np.array(base_pos).repeat(self.block_stride, axis=0)
    
    def loading_seq(self):
        _, self.seq, _, self.qual, _ = self.read.get_analysis_dataset(
            self.basecall_analysis, 'BaseCalled_template/Fastq').split('\n')

    def get_template_signal(self):
        return self.current_signal

    def get_signal_of_seq_loc(self,start,end):
        # TODO: how to using move table and current signal to get the signal of the seq
        _move_start = self.sequence_length-end-1 
        _move_end = self.sequence_length-start 
        return self.current_signal[self.move_list[_move_start]:self.move_list[_move_end]]

    def return_data(self):
        return self.current_signal,self.move_list,self.sequence_length

    def __repr__(self):
        return f'GuppyCalledRead: {self.read.get_read_id()}'


        


In [ ]:
#| export

def get_signal_of_seq_loc(current_signal,move_list,sequence_length,start,end):
    _move_start = sequence_length-end-1 
    _move_end = sequence_length-start 
    return current_signal[move_list[_move_start]:move_list[_move_end]]  
